In [2]:
# used to get environment variables with config info
import os
import time
# used to read the training data CSV files
import pandas as pd
# used for shaping data
import numpy as np

# used to map our activity categories to values
from sklearn.preprocessing import OneHotEncoder

# used to make an ML model
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense

# used to read data from a Kafka topic for inferencing
from tensorflow_io.kafka import KafkaDataset
from tensorflow.io import decode_csv
from sklearn.model_selection import train_test_split
# used to send the inference events to an output topic
from kafka3 import KafkaProducer, KafkaConsumer
import json

# ---------------------------------------------------------------------------

# KAFKA_BOOTSTRAP = '127.0.0.1:9092'
KAFKA_BOOTSTRAP = 'localhost:9092'
RAW_EVENTS_TOPIC = 'RAW_SENSORS'
PROCESSED_EVENTS_TOPIC = 'IPHONE.ACTIVITY'
print("Using Kafka cluster at %s" % (KAFKA_BOOTSTRAP))
print("Reading sensor events from %s and producing predictions to %s" % (RAW_EVENTS_TOPIC, PROCESSED_EVENTS_TOPIC))

import tensorflow as tf
import tensorflow_io as tfio
import pandas as pd
import numpy as np

# columns in the CSV files used in this project
columns = [
    # accelerometer data
    "accel_x",
    "accel_y",
    "accel_z",
    # gyroscope data
    "gyro_x",
    "gyro_y",
    "gyro_z"
]


labels = [
    # phone is on the table
    #  it could be face up or face down, but the point
    #  is that it's not being held, touched or used
    "idle",
    # phone is in your hand and is in use
    #  it could be that you're just looking at it, or
    #  you're actively tapping on it
    "inhand",
    # phone is in a pocket, while you are standing or
    #  walking around
    "pocketmoving",
    # phone is in a pocket, while you are sitting down
    "pocketsitting",
    # phone is in your hand, and you're running!
    "running"
]

Using Kafka cluster at localhost:9092
Reading sensor events from RAW_SENSORS and producing predictions to IPHONE.ACTIVITY


In [450]:
consumer = KafkaConsumer(bootstrap_servers=KAFKA_BOOTSTRAP)
consumer.subscribe([RAW_EVENTS_TOPIC])

In [451]:
message = consumer.poll(timeout_ms=5000)

In [452]:
# Load the saved model
loaded_model = tf.keras.models.load_model("model.h5")

2023-07-05 00:19:32.469542: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-05 00:19:32.471454: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-05 00:19:32.472688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [460]:
len(list(message.items())[0][1])

1

In [462]:
json.loads(list(message.items())[0][1][0].value.decode('utf-8'))['value']

[[0.57462597,
  -0.40223816,
  9.7494869,
  0.00057115901,
  0.0011588114,
  -0.0025497514],
 [0.57462597,
  -0.40223816,
  9.7494869,
  0.00057115901,
  0.0011588114,
  -0.0025497514],
 [0.57462597,
  -0.40223816,
  9.7494869,
  0.00057115901,
  0.0011588114,
  -0.0025497514],
 [0.57462597,
  -0.40223816,
  9.7494869,
  0.00057115901,
  0.0011588114,
  0.0029480357],
 [0.57223165,
  -0.39744961,
  9.7111788,
  -0.00065057148,
  0.00054794608,
  -0.0019388862],
 [0.57702023,
  -0.40463245,
  9.6513214,
  0.0011820243,
  -0.0018955149,
  -0.0019388862],
 [0.57702023,
  -0.40463245,
  9.6513214,
  0.0011820243,
  -0.0018955149,
  -0.0019388862],
 [0.57702023,
  -0.40463245,
  9.7183619,
  0.00057115901,
  -0.00067378435,
  -0.00010629055],
 [0.5913859,
  -0.38787252,
  9.6896305,
  0.00057115901,
  0.00054794608,
  -0.0013280211],
 [0.56744313,
  -0.38787252,
  9.6896305,
  0.00057115901,
  0.00054794608,
  -0.0013280211],
 [0.56744313,
  -0.38787252,
  9.6896305,
  0.00057115901,
  0.00

In [473]:
[json.loads(list(message.items())[0][1][i].value.decode('utf-8'))['timestamp'] for i in range(0, len(list(message.items())[0][1]))]

['2023-07-05 00:19:28']

In [471]:
timestamp_payload = [json.loads(list(message.items())[0][1][i].value.decode('utf-8'))['value'] for i in range(0, len(list(message.items())[0][1]))]

input_payload = np.reshape(np.array(timestamp_payload), (1, 40, 6))

predicted_value = loaded_model.predict(input_payload, verbose=0)

predicted_value

array([[9.99238610e-01, 5.42562226e-08, 2.17743454e-05, 7.39532115e-04,
        1.07473134e-07]], dtype=float32)

In [184]:
[json.loads(list(message.items())[0][1][i].value.decode('utf-8'))['timestamp'] for i in range(0, 1)]

['2023-07-04 21:20:17']

In [405]:
list(message.items())[0][1][0]

ConsumerRecord(topic='RAW_SENSORS', partition=0, offset=29331, timestamp=1688512014897, timestamp_type=0, key=None, value=b'{"value": [[0.57702023, -0.36632404, 9.5196371, -0.00080634211, -0.00089675013, 0.00043860124], [0.57702023, -0.36632404, 9.7375154, 0.0016371189, -0.00089675013, 0.0028820622], [0.57702023, -0.37111259, 9.7087841, -0.00019547688, -0.00028588495, 0.0028820622], [0.56983739, -0.37590113, 9.6896305, 0.0010262536, -0.0015076154, 0.0010494665], [0.58180881, -0.39026681, 9.7135725, -0.00080634211, -0.00028588495, 0.00043860124], [0.56983739, -0.39026681, 9.7135725, -0.00080634211, -0.00028588495, 0.0010494665], [0.57223165, -0.36871833, 9.7135725, -0.00019547688, 0.00032498033, -0.00078312919], [0.59378016, -0.3998439, 9.7399101, 0.00041538838, -0.00028588495, -0.0026157252], [0.58420306, -0.40702671, 9.7063904, 0.0016371189, -0.00089675013, 0.0028820622], [0.57702023, -0.38308397, 9.7423038, 0.00041538838, 0.0033793065, -0.00078312919], [0.59617442, -0.3998439, 9.7087

In [429]:
np.array(json.loads(list(message.items())[0][1][0].value.decode('utf-8'))['value'])

array([[ 5.7702023e-01, -3.6632404e-01,  9.5196371e+00, -8.0634211e-04,
        -8.9675013e-04,  4.3860124e-04],
       [ 5.7702023e-01, -3.6632404e-01,  9.7375154e+00,  1.6371189e-03,
        -8.9675013e-04,  2.8820622e-03],
       [ 5.7702023e-01, -3.7111259e-01,  9.7087841e+00, -1.9547688e-04,
        -2.8588495e-04,  2.8820622e-03],
       [ 5.6983739e-01, -3.7590113e-01,  9.6896305e+00,  1.0262536e-03,
        -1.5076154e-03,  1.0494665e-03],
       [ 5.8180881e-01, -3.9026681e-01,  9.7135725e+00, -8.0634211e-04,
        -2.8588495e-04,  4.3860124e-04],
       [ 5.6983739e-01, -3.9026681e-01,  9.7135725e+00, -8.0634211e-04,
        -2.8588495e-04,  1.0494665e-03],
       [ 5.7223165e-01, -3.6871833e-01,  9.7135725e+00, -1.9547688e-04,
         3.2498033e-04, -7.8312919e-04],
       [ 5.9378016e-01, -3.9984390e-01,  9.7399101e+00,  4.1538838e-04,
        -2.8588495e-04, -2.6157252e-03],
       [ 5.8420306e-01, -4.0702671e-01,  9.7063904e+00,  1.6371189e-03,
        -8.9675013e-04, 

In [2]:
NUM_COLUMNS = len(columns)
BATCH_SIZE=64
# size of each window - the number of samples in each value
WINDOW_SIZE = 40
# how often to start a new window
WINDOW_STEP = 15

In [424]:
# Load the saved model
loaded_model = tf.keras.models.load_model("model.h5")

2023-07-05 00:09:23.640268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-05 00:09:23.641408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-05 00:09:23.642383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [431]:
payload = np.array(json.loads(list(message.items())[0][1][0].value.decode('utf-8'))['value'])

loaded_model.predict(np.reshape(np.array(payload), (1, 40, 6)))

1/1 [==============================] - 0s 27ms/step


array([[9.9924171e-01, 5.4289931e-08, 2.1803358e-05, 7.3638331e-04,
        1.0746240e-07]], dtype=float32)

In [126]:
while True:

    consumer = KafkaConsumer(bootstrap_servers=KAFKA_BOOTSTRAP)

    consumer.subscribe([RAW_EVENTS_TOPIC])

    payload = consumer.poll(timeout_ms=500)

    array_payload = [json.loads(list(message.items())[0][1][0].value.decode('utf-8'))['value'] for i in range(0,40)]

    timestamp_payload = [json.loads(list(message.items())[0][1][0].value.decode('utf-8'))['timestamp'] for i in range(0,1)]

    input_payload = np.reshape(np.array(array_payload), (1, 40, 6))

    predicted_value = loaded_model.predict(input_payload, verbose=0)

    idx_predicted = np.argmax(predicted_value)

    print('**************************************')
    print(timestamp_payload, labels[idx_predicted])
    print('**************************************')

    

**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
**************************************
['2023-07-04 20:41:22'] idle
**************************************
*************************************

KeyboardInterrupt: 

In [39]:
import tensorflow_io as tfio

KAFKA_BOOTSTRAP = 'localhost:9092'
RAW_EVENTS_TOPIC = 'RAW_SENSORS'


# payload = tfio.experimental.streaming.KafkaGroupIODataset(
payload = tfio.IODataset.from_kafka(
    topic=RAW_EVENTS_TOPIC,
    group_id="testcg",
    servers=KAFKA_BOOTSTRAP,
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=latest"
    ],
)

# # translate each of the string messages to an array of 6 numbers
# payload = payload.map(decode_kafka_message)


payload = payload.batch(WINDOW_SIZE)

2023-07-04 16:16:03.296669: I tensorflow_io/core/kernels/kafka_kernels.cc:107] Kafka configuration: session.timeout.ms=7000
2023-07-04 16:16:03.296719: I tensorflow_io/core/kernels/kafka_kernels.cc:107] Kafka configuration: max.poll.interval.ms=8000
2023-07-04 16:16:03.296744: I tensorflow_io/core/kernels/kafka_kernels.cc:107] Kafka configuration: auto.offset.reset=latest
2023-07-04 16:16:03.296766: I tensorflow_io/core/kernels/kafka_kernels.cc:107] Kafka configuration: bootstrap.servers=localhost:9092
2023-07-04 16:16:03.801108: I tensorflow_io/core/kernels/kafka_kernels.cc:349] Kafka tail: 1928


In [40]:
tensor = iter(payload)

2023-07-04 16:16:06.022814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]
2023-07-04 16:16:06.023442: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]


In [42]:
count = 0

while True:

    try:
        optional = tensor.get_next_as_optional()

        if optional.has_value().numpy():
            raw_payload = tensor.get_next()[0]

            tmp_payload = [json.loads(raw_payload.numpy()[i].decode('utf-8'))['value'] for i in range(0, 40)]

            timestamp_payload = [json.loads(raw_payload.numpy()[i].decode('utf-8'))['timestamp'] for i in range(0, 1)]

            input_payload = np.reshape(np.array(tmp_payload), (1, 40, 6))

            predicted_value = loaded_model.predict(input_payload)

            idx_predicted = np.argmax(predicted_value)

            count = count + 1

            print(f'timestamp={timestamp_payload} and label={labels[idx_predicted]}')

    except InterruptedError as e:
        if e == 'OutOfRangeError':
            print('OutOfRangeError')
            print(e, count)
            break

KeyboardInterrupt: 

: 